In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
from google.colab import drive
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import joblib
# Download necessary NLTK data
nltk.download('vader_lexicon')
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
df=pd.read_csv('/content/drug_data.csv')

Please enter your drug review: I used skyla it worked well

Top words for each topic:
Topic 1: skyla, used, worked, well
Topic 2: well, worked, skyla, used
Topic 3: used, skyla, worked, well

Topic distribution for the review:
Topic 1: 0.23
Topic 2: 0.62
Topic 3: 1.33

Dominant topic: Topic 3

In [23]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

def preprocess_text(text):
    # Convert to lowercase and remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())

    # Tokenize and remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)

    # Add domain-specific stopwords
    domain_stopwords = {'mg', 'tablet', 'capsule', 'pill', 'dose', 'doctor', 'prescription'}
    stop_words.update(domain_stopwords)

    return ' '.join([word for word in tokens if word not in stop_words])

def peform_topic_modeling(text):
    # Preprocess the user's review
    processed_review = preprocess_text(text)

    # Create TF-IDF matrix
    vectorizer = TfidfVectorizer(max_df=1.0, min_df=1)  # Adjust for single document
    tfidf_matrix = vectorizer.fit_transform([processed_review])

    # Apply NMF
    num_topics = 3  # Using a small number of topics for a single review
    nmf_model = NMF(n_components=num_topics, random_state=42)
    nmf_output = nmf_model.fit_transform(tfidf_matrix)

    feature_names = vectorizer.get_feature_names_out()
    num_top_words = 5

    all_top_words = []
    topic_distributions = []

    for topic_idx, topic in enumerate(nmf_model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]
        all_top_words.append(f"Topic {topic_idx + 1}: {', '.join(top_words)}")

    for i, topic_prob in enumerate(nmf_output[0]):
        topic_distributions.append(f"Topic {i+1}: {topic_prob:.2f}")

    dominant_topic = nmf_output[0].argmax() + 1
    dominant_topic_text = f"Dominant topic: Topic {dominant_topic}"

    return ' '.join(all_top_words), ' '.join(topic_distributions), dominant_topic_text

# Get user input
user_review = input("Please enter your drug review: ")

a, b, c = peform_topic_modeling(user_review)
print(a)
print(b)
print(c)


Please enter your drug review:  I used skyla it worked well
Topic 1: skyla, used, worked, well Topic 2: well, worked, skyla, used Topic 3: used, skyla, worked, well
Topic 1: 0.23 Topic 2: 0.62 Topic 3: 1.33
Dominant topic: Topic 3
